In [29]:
# model and data are ready
# it's time to train, validate and test the model
# training is an iterative process, in each iteration
# > the model guesses an output 
# > calculates the error with the guess through loss function
# > collects the derivatives of error (retrieve the gradients of parameters)
# > optimizes parameters using the gradient descent

In [30]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [31]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [32]:
learning_rate = 1e-3
batch_size = 64

In [37]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # set mode as training: important for batch normalization and dropout layers
    model.train() # unnecessary in this situation, but added for best practice
    for batch, (X, y) in enumerate(dataloader):
        # forward
        pred = model(X)
        loss = loss_fn(pred, y)

        # back propagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch%100 == 0: # print every 100 batches
            loss, current = loss.item(), batch*batch_size + len(X)
            # print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    # set mode as evaluation: important for batch normalization and dropout layers
    model.eval() # unnecessary in this situation, but added for best practice
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # disable gradient computing
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test --- Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")


In [ ]:
# the main
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10000 # read as epics, means an entire training dataset through the learning algorithm
for t in range(epochs):
    # print(f"Epoch {t+1}\n" + '-'*20)
    print(f"Epoch {t+1} ", end='')
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

# did 10+100+1000+10000 epochs, got Accuracy: 88.8%, Avg loss: 0.925029

Epoch 1 Test --- Accuracy: 88.8%, Avg loss: 0.392407

Epoch 2 Test --- Accuracy: 88.8%, Avg loss: 0.392732

Epoch 3 Test --- Accuracy: 88.8%, Avg loss: 0.393123

Epoch 4 Test --- Accuracy: 88.8%, Avg loss: 0.393309

Epoch 5 Test --- Accuracy: 88.8%, Avg loss: 0.393461

Epoch 6 Test --- Accuracy: 88.7%, Avg loss: 0.393496

Epoch 7 Test --- Accuracy: 88.7%, Avg loss: 0.393955

Epoch 8 Test --- Accuracy: 88.8%, Avg loss: 0.394323

Epoch 9 Test --- Accuracy: 88.7%, Avg loss: 0.394221

Epoch 10 Test --- Accuracy: 88.7%, Avg loss: 0.394786

Epoch 11 Test --- Accuracy: 88.8%, Avg loss: 0.394785

Epoch 12 Test --- Accuracy: 88.7%, Avg loss: 0.395461

Epoch 13 Test --- Accuracy: 88.7%, Avg loss: 0.395311

Epoch 14 Test --- Accuracy: 88.7%, Avg loss: 0.395470

Epoch 15 Test --- Accuracy: 88.7%, Avg loss: 0.395670

Epoch 16 Test --- Accuracy: 88.7%, Avg loss: 0.395958

Epoch 17 Test --- Accuracy: 88.7%, Avg loss: 0.396295

Epoch 18 Test --- Accuracy: 88.7%, Avg loss: 0.396470

Epoch 19 Test --- A

In [ ]:
# save the model
torch.save(model.state_dict(), "5_weights.pth")